In [20]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

path = '../Data/006/Lev5/' #path for the simulation
extrapolation_order = 2

alm = []
H2 = []

input_file1 = h5py.File(path+'rhOverM_Asymptotic_GeometricUnits.h5','r')
SXS = input_file1['Extrapolated_N%i.dir'%(extrapolation_order)]['Y_l%i_m%i.dat'%(2,-1)]
t = SXS[:,0]
dt = np.diff(t)

def ALM(l,m):
    #print("lol",l,m)
    if m>l or m<-l or l<2 or l>8:
        return 0
    return alm[l-2][m+l]
def Hcomp(l,m):
    if m>l or m<-l or l<2 or l>8:
        return 0
    return H2[l-2][m+l]

meta = open(path+'metadata.txt','r')
for line in meta:
    line = line.translate(None,",")
    line = line.split()
    if(len(line)<2):
        continue;
    var = line[0].translate(None," ")
    if var == 'initial-ADM-energy':
        initial_mass = float(line[2])
    if var == 'initial-ADM-linear-momentum':
        initial_p = np.array([float(line[2]),float(line[3]),float(line[4])])
    if var == 'initial-ADM-angular-momentum':
        initial_j = np.array([float(line[2]),float(line[3]),float(line[4])])
    if var == 'remnant-mass':
        remnant_mass = float(line[2])
    if var == 'remnant-spin':
        remnant_spin = np.array([float(line[2]),float(line[3]),float(line[4])])

for l in range(2,9):
    alm.append([])
    H2.append([])
    for m in range(-l,l+1):
        SXS = input_file1['Extrapolated_N%i.dir'%(extrapolation_order)]['Y_l%i_m%i.dat'%(l,m)]
        H = SXS[:,1]+(0+1j)*SXS[:,2]
        t = SXS[:,0]
        Hdot = np.diff(H)/np.diff(t)
        alm[l-2].append(Hdot)
        H2[l-2].append(H[1:])
        
#The following are coefficients used in calculations of radiated angular and linear momentum       
def f(l,m):
    return np.sqrt((l-m)*(l+m+1))
def a(l,m):
    return np.sqrt((1.0+0j)*(l-m)*(l+m+1.0))/(l*(l+1.0))
def b(l,m):
    return (np.sqrt((1+0j)*(l-2.0)*(l+2.0)*(l+m)*(l+m-1.0)/(2.0*l-1.0)/(2.0*l+1.0)))/(2.0*l)
def c(l,m):
    return 2.0*m/(l*(l+1.0))
def d(l,m):
    return (np.sqrt((1.0+0j)*(l-2.0)*(l+2.0)*(l-m)*(l+m)/(2.0*l-1.0)/(2.0*l+1)))/l

for l in range(2,9):
    alm.append([])
    H2.append([])
    for m in range(-l,l+1):
        SXS = input_file1['Extrapolated_N2.dir']['Y_l%i_m%i.dat'%(l,m)]
        H = SXS[:,1]+(0+1j)*SXS[:,2]
        t = SXS[:,0]
        Hdot = np.diff(H)/np.diff(t)
        alm[l-2].append(Hdot)
        H2[l-2].append(H[1:])



        
    

    

In [21]:
dp_plus = (0+0j)*(dt)
dp_z = dp_plus
djx = dp_plus
djy = djx
djz = djx
de = djx
for l in range(2,9):
    for m in range(-l,l+1):
        djx = djx + np.imag(Hcomp(l,m)*(f(l,m)*np.conj(ALM(l,m+1)) + f(l,-m)*np.conj(ALM(l,m-1))))
        djy = djy - np.real(Hcomp(l,m)*(f(l,m)*np.conj(ALM(l,m+1)) - f(l,-m)*np.conj(ALM(l,m-1))))
        djz = djz + np.imag(m*Hcomp(l,m)*np.conj(ALM(l,m)))
        de = de + ALM(l,m)*np.conj(ALM(l,m))
        dp_plus = dp_plus+ALM(l,m)*(a(l,m)*np.conj(ALM(l,m+1)) + b(l,-m)*np.conj(ALM(l-1,m+1)) - b(l+1,m+1)*np.conj(ALM(l+1,m+1)))
        dp_z = dp_z + ALM(l,m)*(c(l,m)*np.conj(ALM(l,m)) + d(l,m)*np.conj(ALM(l-1,m)) + d(l+1,m)*np.conj(ALM(l+1,m)))

In [22]:
JX = np.cumsum(djx*dt)/(32*np.pi)
JY = np.cumsum(djy*dt)/(32*np.pi)
JZ = np.cumsum(djz*dt)/(16*np.pi)
E = np.cumsum(de*dt)/(16*np.pi)
p_plus = np.cumsum(dp_plus*dt)/(8*np.pi)
p_z = np.cumsum(dp_z*dt)/(16*np.pi)
p_x = np.real(p_plus)
p_y = np.imag(p_plus)


final_mass = initial_mass - E[-1]
print(remnant_mass,final_mass,(remnant_mass-final_mass)/remnant_mass)

radiation_spin = np.array([JX[-1],JY[-1],JZ[-1]])
final_spin = initial_j - radiation_spin
print(remnant_spin,final_spin)
print((remnant_spin-final_spin)/remnant_spin)
Jsq_init= (remnant_spin[0]*remnant_spin[0]+remnant_spin[1]*remnant_spin[1])
Jsq_final = final_spin[0]*final_spin[0]+final_spin[1]*final_spin[1]
print((Jsq_init-Jsq_final)/(Jsq_init))

(0.956545323918, (0.95648006344574177+0j), (6.8225175144740865e-05+0j))
(array([ 0.04660945,  0.02758132,  0.57907841]), array([ 0.04661495,  0.0275272 ,  0.57943781]))
[-0.00011786  0.00196226 -0.00062063]
0.000842249550424
